In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re

In [2]:
salariesBycollegeCSV = "Resources/DS_WhereItPaysToAttendCollege/salaries-by-college-type.csv"
salariesBycollege_df = pd.read_csv(salariesBycollegeCSV)

college_AccTuitionCSV = "Resources/DS_USWorldReportCollegeData/College.csv"
college_AccTuition_df = pd.read_csv(college_AccTuitionCSV)

usa_rankingCSV = "Resources/NationalUniversitiesRankings.csv"
usa_ranking_df = pd.read_csv(usa_rankingCSV, encoding = "latin1")

In [3]:
salariesBycollege_df.head()
#School Name, Starting Median Salary, Mid-Career Median Salary

salariesBycollege_df.rename(columns={'School Name':'Name'}, inplace=True)

salariesBycollege_df = salariesBycollege_df.loc[:, ['Name', 'Starting Median Salary', 'Mid-Career Median Salary']]
salariesBycollege_df.head()

#salariesBycollege_df.describe #[269 rows x 3 columns]>


,Name,Starting Median Salary,Mid-Career Median Salary
0,Massachusetts Institute of Technology (MIT),"$72,200.00","$126,000.00"
1,California Institute of Technology (CIT),"$75,500.00","$123,000.00"
2,Harvey Mudd College,"$71,800.00","$122,000.00"
3,"Polytechnic University of New York, Brooklyn","$62,400.00","$114,000.00"
4,Cooper Union,"$62,200.00","$114,000.00"


In [4]:
#replaces () with nothing.. reference here: 
#https://stackoverflow.com/questions/20894525/how-to-remove-parentheses-and-all-data-within-using-pandas-python

for i, row in salariesBycollege_df.iterrows():

    salariesBycollege_df.loc[i, 'Name'] = re.sub(r" ?\([^)]+\)", "", salariesBycollege_df.loc[i, 'Name'])
    
salariesBycollege_df.head()

,Name,Starting Median Salary,Mid-Career Median Salary
0,Massachusetts Institute of Technology,"$72,200.00","$126,000.00"
1,California Institute of Technology,"$75,500.00","$123,000.00"
2,Harvey Mudd College,"$71,800.00","$122,000.00"
3,"Polytechnic University of New York, Brooklyn","$62,400.00","$114,000.00"
4,Cooper Union,"$62,200.00","$114,000.00"


In [5]:

#name, Private, Apps, Accept "AcpRate"
college_AccTuition_df.rename( columns={'Unnamed: 0':'Name'}, inplace=True )

college_accept_df = college_AccTuition_df.loc[:, ['Name', 'Private', 'Apps', 'Accept']]

college_accept_df['Acp_Rate(%)'] = round(college_accept_df['Accept'] / college_accept_df['Apps'] * 100,2)

college_accept_df.head()



,Name,Private,Apps,Accept,Acp_Rate(%)
0,Abilene Christian University,Yes,1660,1232,74.22
1,Adelphi University,Yes,2186,1924,88.01
2,Adrian College,Yes,1428,1097,76.82
3,Agnes Scott College,Yes,417,349,83.69
4,Alaska Pacific University,Yes,193,146,75.65


In [6]:
new1_merged_df = usa_ranking_df.merge(salariesBycollege_df, on='Name', how='left')

#new1_merged_df
new1_merged_df.isnull().sum()

Name                          0
Location                      0
Rank                          0
Description                   0
Tuition and fees              0
In-state                     98
Undergrad Enrollment          0
Starting Median Salary      157
Mid-Career Median Salary    157
dtype: int64

In [7]:
usa_ranking_df.head()
#Name, Location, Rank, Tuition and fees
usa_ranking_df = usa_ranking_df.loc[:,['Name', 'Location', 'Rank', 'Tuition and fees']]
usa_ranking_df.head()



,Name,Location,Rank,Tuition and fees
0,Princeton University,"Princeton, NJ",1,"$45,320"
1,Harvard University,"Cambridge, MA",2,"$47,074"
2,University of Chicago,"Chicago, IL",3,"$52,491"
3,Yale University,"New Haven, CT",3,"$49,480"
4,Columbia University,"New York, NY",5,"$55,056"


In [8]:
merged_df = usa_ranking_df.merge(college_accept_df, on='Name', how='left')
merged_df.head()
#merged_df.count()

,Name,Location,Rank,Tuition and fees,Private,Apps,Accept,Acp_Rate(%)
0,Princeton University,"Princeton, NJ",1,"$45,320",Yes,13218.0,2042.0,15.45
1,Harvard University,"Cambridge, MA",2,"$47,074",Yes,13865.0,2165.0,15.61
2,University of Chicago,"Chicago, IL",3,"$52,491",Yes,6348.0,2999.0,47.24
3,Yale University,"New Haven, CT",3,"$49,480",Yes,10705.0,2453.0,22.91
4,Columbia University,"New York, NY",5,"$55,056",Yes,6756.0,1930.0,28.57


In [9]:
merged_df.isnull().sum()

Name                  0
Location              0
Rank                  0
Tuition and fees      0
Private             121
Apps                121
Accept              121
Acp_Rate(%)         121
dtype: int64

In [10]:
#get list of schools with NaN values

missingData = []
special_df = merged_df.isnull().any(axis=1)
#print(special_df)

for i, row in enumerate(special_df):
    if row == True:
        #print(i,row)
        
        for x, stuff in merged_df.iterrows():
            if x == i:
                #print(stuff['Name'])
                missingData.append(stuff['Name'])

In [11]:
#bring in CSV for grabbing missing Acceptance rate for NaN values
#mostRecentDataCSV = "Resources/Most-Recent-Cohorts-All-Data-Elements.csv"
mostRecentDataCSV = "Resources/DS_collegescorecard_ed_gov/Most-Recent-Cohorts-All-Data-Elements.csv"
mostRecentData_df = pd.read_csv(mostRecentDataCSV,error_bad_lines=False, index_col=False, dtype='unicode')
#mostRecentData_df.columns

Data improvement form another resource in effort to reduce the amount of "NaN" values in "merged_USR_cacc_df = merged_df"

In [13]:
#using school name from missingData grab acceptance rate from mostRecentData_df[ADM_RATE_ALL'] 

missingADMrate = {}

mostADMrate_MDsalary_df = mostRecentData_df.loc[:,['INSTNM','ADM_RATE_ALL','MD_EARN_WNE_P10']]
mostADMrate_MDsalary_df.head()

for i, row in mostADMrate_MDsalary_df.iterrows():

    for line in missingData:
        
        if row[0] == line:
            row[1] = float(row[1])
            row[1] = round(row[1] * 100,2)
            missingADMrate.update({line:row[1]})
            
if 'Benedictine University' in missingADMrate: 
    del missingADMrate['Benedictine University']
            
#for (key, value) in missingADMrate.items():
    #print(key,"::", value)
    

In [14]:
#change name of merged_df to relect the two df merged into it... and eleminate a few columns

merged_USR_cacc_df = merged_df

#if Acp_Rate(%) has a NaN value and a new value in missingADMrate, map and replace the NaN value
merged_USR_cacc_df.loc[merged_USR_cacc_df['Acp_Rate(%)'].isnull(), 'Acp_Rate(%)'] = merged_USR_cacc_df['Name'].map(missingADMrate)

merged_USR_cacc_df = merged_USR_cacc_df.loc[:,['Name','Location','Rank','Tuition and fees','Private','Acp_Rate(%)']]
#merged_USR_cacc_df.head(10)


In [15]:
#slight improvement in the Acceptance rate NaN values

merged_USR_cacc_df.isnull().sum()


Name                  0
Location              0
Rank                  0
Tuition and fees      0
Private             121
Acp_Rate(%)          73
dtype: int64

In [16]:
#make final merge 
finalmerge_df = merged_USR_cacc_df.merge(salariesBycollege_df, on='Name', how='left')
finalmerge_df.isnull().sum()

Name                          0
Location                      0
Rank                          0
Tuition and fees              0
Private                     124
Acp_Rate(%)                  74
Starting Median Salary      157
Mid-Career Median Salary    157
dtype: int64

In [17]:
#get list of schools with NaN values.... going to work on improving Salary columns

missingSalaryDatalist = []

missingSalaryData = pd.isnull(finalmerge_df['Starting Median Salary'])

for i, row in enumerate(missingSalaryData):
    if row == True:
        
        for x, stuff in finalmerge_df.iterrows():
            if x == i:
                #print(stuff['Name'])
                missingSalaryDatalist.append(stuff['Name'])
                
print(len(missingSalaryDatalist))

157


In [20]:
#Bring in the next resourse for the Salary info:
#using school name from missingMDsalary grab salary info from mostADMrate_MDsalary_df['MD_EARN_WNE_P10'] 

missingMDsalary = {}

#mostMDsalary_df = mostRecentScorecard_df.loc[:,['INSTNM','MD_EARN_WNE_P10']]
#mostMDsalary_df.head()
#mostADMrate_MDsalary_df

for i, row in mostADMrate_MDsalary_df.iterrows():

    for line in missingSalaryDatalist:
        
        if row[0] == line:
            missingMDsalary.update({line:row[2]})
            
#for (key, value) in missingMDsalary.items():
#    print(key,"::", value)



In [21]:
#repalce NaN value in Mid-Career Median Salary column with new data from missingMDsalary
finalmerge_df.loc[finalmerge_df['Mid-Career Median Salary'].isnull(), 'Mid-Career Median Salary'] = finalmerge_df['Name'].map(missingMDsalary)  

finalmerge_df.isnull().sum()



Name                          0
Location                      0
Rank                          0
Tuition and fees              0
Private                     124
Acp_Rate(%)                  74
Starting Median Salary      157
Mid-Career Median Salary     71
dtype: int64